In [2]:
import numpy as np
import h5py
import torch
#from torch.utils.data import Dataset, DataLoader
# import torch.utils.data as data
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

/Users/wilsonchang/Desktop/Project/test/.env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Helper Function

In [3]:
class EEGDataset(data_utils.Dataset):
    """EEG dataset."""
    def __init__(self, X, y):
        self.X = X;
        self.y = y;
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
class Flatten(nn.Module):
    def forward(self, x):
        N, E, T = x.size() 
        return x.view(N, -1)

# Preprocess the Data

In [12]:
A01T = h5py.File('A01T_slice.mat', 'r')
print (A01T)
X = np.copy(A01T['image'])
y = np.copy(A01T['type'])
y = y[0,0:X.shape[0]:1]
y = np.asarray(y, dtype=np.int32)
y = y - 769;

<HDF5 file "A01T_slice.mat" (mode r)>


In [18]:
data = EEGDataset(X,y)
dset = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [len(data)-100, 50, 50])
dataloaders = {x: data_utils.DataLoader(dset[x], batch_size=10, shuffle=True, num_workers=1) for x in ['train', 'val', 'test']}
dataiter = iter(dataloaders['train'])
X_train, y_train = dataiter.next()
print(y_train)


 0
 3
 0
 1
 1
 2
 1
 0
 0
 2
[torch.IntTensor of size 10]



# Testing the Dataloader

In [10]:
# Torch.DoubleTensor
for i_batch, sample_batched in enumerate(dataloaders['train']):
    X, y = sample_batched
    print (y)


 2
 0
 0
 0
 1
 3
 0
 1
 2
 1
[torch.IntTensor of size 10]


 2
 0
 0
 3
 2
 2
 2
 3
 3
 0
[torch.IntTensor of size 10]


 0
 0
 2
 0
 3
 1
 3
 0
 1
 0
[torch.IntTensor of size 10]


 3
 3
 1
 0
 2
 3
 2
 1
 0
 3
[torch.IntTensor of size 10]


 3
 3
 3
 1
 2
 2
 0
 3
 3
 3
[torch.IntTensor of size 10]


 1
 3
 1
 0
 1
 2
 0
 2
 0
 3
[torch.IntTensor of size 10]


 1
 2
 1
 0
 3
 2
 0
 2
 2
 2
[torch.IntTensor of size 10]


 3
 3
 0
 2
 1
 1
 0
 0
 2
 1
[torch.IntTensor of size 10]


 2
 1
 0
 2
 3
 0
 1
 0
 2
 2
[torch.IntTensor of size 10]


 1
 3
 0
 3
 3
 0
 2
 2
 2
 1
[torch.IntTensor of size 10]


 1
 3
 3
 0
 3
 2
 0
 1
 3
 2
[torch.IntTensor of size 10]


 0
 0
 1
 0
 1
 1
 1
 0
 3
 3
[torch.IntTensor of size 10]


 2
 3
 2
 1
 1
 1
 1
 0
 2
 2
[torch.IntTensor of size 10]


 0
 2
 3
 1
 2
 0
 3
 3
 1
 2
[torch.IntTensor of size 10]


 0
 2
 0
 1
 1
 3
 3
 1
 0
 2
[torch.IntTensor of size 10]


 1
 1
 1
 3
 3
 2
 1
 2
 1
 2
[torch.IntTensor of size 10]


 3
 1
 0
 0
 3
 1
 2
 1

# Define the Model

In [7]:
dtype = torch.DoubleTensor
#N, E, T= 100, 25, 500
#C = 4
N, E, T= 288, 25, 1000
C = 4;
model = nn.Sequential(
    nn.LSTM(E,C,2)
)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Test Training

In [9]:
for i_batch, sample_batched in enumerate(dataloaders['train']):
        X_train, y_train = sample_batched
        print (X_train)
        X_train = Variable(X_train)
        y_train = Variable(y_train.long())
        #dtype = torch.FloatTensor
        #X_train.type(dtype)
        #y_train.type(dtype)
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train.type(torch.LongTensor))
        print('(%d batch) loss: %f' % (i_batch, loss))
        # Do backpropagation
        model.zero_grad()
        loss.backward()
        # Update the parameters
        optimizer.step()


( 0  ,.,.) = 
   13.4277   18.0664    5.4199  ...     8.3008   -2.2949    5.7129
   11.3281   15.1367    5.1270  ...    -2.1484  -11.0352   -3.0762
    9.2773   13.5742    2.1973  ...     4.1992   -6.5918    0.7812
              ...                ⋱                ...             
    5.3711   12.6953    2.4414  ...     6.8359    1.4648   12.2070
   15.6250   21.9727    6.8359  ...    11.7188    6.3477   17.0898
   10.2539   15.1367    0.4883  ...    11.2305    6.8359   19.5312

( 1  ,.,.) = 
   19.7266    9.7656    8.9355  ...    10.0586   11.9141   13.0859
   25.0977   15.1367   13.9648  ...    15.1855   17.6270   14.4043
   20.9473   12.0605   11.4746  ...    10.2539   13.1836   14.2578
              ...                ⋱                ...             
   20.9961   10.2539   13.1836  ...    12.2070    8.7891    9.2773
   29.2969   17.0898   14.1602  ...    13.6719   13.1836   11.7188
   14.6484    8.7891    4.3945  ...     7.8125    8.7891   -0.4883

( 2  ,.,.) = 
   -9.5215  -10.0

NameError: name 'fn' is not defined

# Training

In [16]:
num_epochs = 2
for epoch in range(num_epochs):  # loop over the dataset multiple times
    for i_batch, sample_batched in enumerate(dataloaders['train']):
        X_train, y_train = sample_batched
        X_train = Variable(X_train)
        y_train = Variable(y_train)
        #dtype = torch.FloatTensor
        #X_train.type(dtype)
        #y_train.type(dtype)
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train.type(torch.LongTensor))
        print('(%d batch) loss: %f' % (i_batch, loss))
        # Do backpropagation
        model.zero_grad()
        loss.backward()
        # Update the parameters
        optimizer.step()
    ## Training Accuracy After 1 epoch
    y_pred_tot = []
    y_train_tot = []
    for i_batch, sample_batched in enumerate(dataloaders['val']):
        X_train, y_train = sample_batched
        X_train, y_train = Variable(X_train), Variable(y_train)
        scores = model(X_train)
        print (scores)
        y_pred_tot.append(np.argmax(scores, axis=0))
        y.train_tot.append(y_train)
    train_acc = np.mean(y_pred_tot == y_train_tot)
    ## Validation Accuracy After 1 epoch    
    y_pred_tot = []
    y_val_tot = []
    for i_batch, sample_batched in enumerate(dataloaders['val']):
        X_val, y_val = sample_batched
        X_val, y_val = Variable(X_val), Variable(y_train)
        scores = model(X_val)
        y_pred_tot.append(np.argmax(scores, axis=0))
        y.val_tot.append(y_val)
    val_acc = np.mean(y_pred_tot == y_val_tot)
    print('(Epoch %d / %d) train acc: %f; val_acc: %f' % (epoch, num_epochs, train_acc, val_acc))


(0 batch) loss: 0.042241
(1 batch) loss: 5.816192
(2 batch) loss: 0.000015
(3 batch) loss: 11.123883
(4 batch) loss: 3.250899
(5 batch) loss: 5.925397
(6 batch) loss: 5.162231
(7 batch) loss: 0.000006
(8 batch) loss: 27.439180
(9 batch) loss: 15.268533
(10 batch) loss: 0.000000
(11 batch) loss: 18.906391
(12 batch) loss: 0.000000
(13 batch) loss: 31.127072
(14 batch) loss: 14.552045
(15 batch) loss: 0.000000
(16 batch) loss: 4.447059
(17 batch) loss: 0.000001
(18 batch) loss: 7.657994
Variable containing:
 222.5675  -73.2677 -135.2047  -40.7436
 184.7829  153.4643  -14.8004 -192.5471
      nan       nan       nan       nan
 -33.8853  128.4333  124.1350 -194.3442
 213.3142  117.0116   91.9718 -125.0277
 -12.5737  147.8971 -177.1536 -107.6746
 154.4318  -88.4074   37.0218   35.3798
 -59.4713 -138.5239   13.0236   37.9769
-150.7744  100.7293 -343.2048   84.3093
-174.9866 -155.7896 -217.7266  225.0662
[torch.DoubleTensor of size 10x4]



SystemError: <built-in method gt of Variable object at 0x10a923408> returned a result with an error set